# Parameters

In [1]:
# IMPORTANT: THE FILE SHOULD ONLY CONTAIN DATES AND BLANK CELLS ON COLUMN "DATE", IF THERE'S SOMETHING ELSE PLEASE ERASE IT
# OTHERWISE THE PROGRAM WON'T RUN PROPERLY MAKE SURE TO CHECK IT BEFORE RUNNING THE CODE.
filename = 'Genomma Labs Logs WO 02-21-22.xlsx'
quarter = 'Q1 2022'

# Libraries

In [2]:
import pandas as pd
import json
import os
from datetime import timedelta
from datetime import datetime
import pyodbc

pd.options.display.float_format = '{:,.2f}'.format

# DWH

## Connections

In [3]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_DWH;'
                     'Trusted_Connection=yes;')

conn2 = pyodbc.connect('Driver={SQL Server};'
                     'Server=SFEDWH01;'
                     'Database=Gnm_MasterOp;'
                     'Trusted_Connection=yes;')

## Queries

In [4]:
query_dates = '''SELECT TmpFecha AS Date, TmpSemanaAnioGenomma, TmpAnioSemanaGenomma
          FROM dbo.Dim_Tiempo'''

query_groups = '''SELECT DISTINCT ProNombre AS AGRUPACION, MrcID, MrcNombre, LinID, LinNombre, ProID
              FROM dbo.vw_EstructuraProductosInternacional WHERE TipoComNombre = 'Estados Unidos de América'
              '''

## Import Data

In [5]:
df_tiempo = pd.read_sql(query_dates, conn1)

df_groups = pd.read_sql(query_groups, conn2)

# Variables

In [6]:
day_numbr = {'Sunday':'S-D',
             'Monday':'L-V',
             'Tuesday':'L-V',
             'Wednesday':'L-V',
             'Thursday':'L-V',
             'Friday':'L-V',
             'Saturday':'S-D'}

day_numbr_n = {'Sunday':1,
               'Monday':2,
               'Tuesday':3,
               'Wednesday':4,
               'Thursday':5,
               'Friday':6,
               'Saturday':7}

month_name = {1:'Enero',
              2:'Febrero',
              3:'Marzo',
              4:'Abril',
              5:'Mayo',
              6:'Junio',
              7:'Julio',
              8:'Agosto',
              9:'Septiembre',
              10:'Octubre',
              11:'Noviembre',
              12:'Diciembre'}

In [7]:
#TV SHOWS OF UNIVISION WITH DIFFERENT DAYPART DON'T FORGET TO KEEP UPDATING THIS LIST
different_dayparts = {'Sports':['Combate Global', 'La Jugada', 'Republica Deportiva',
                                'Contacto Deportivo'],
                      'News' : ['Noticiero Uni Ed Nocturna', 'Noticiero Univision FDS EN',
                                'Noticiero Univision', 'Noticiero Univision FDS'],
                      'Specials':['Desfile de las Rosas', 'Desfile de las Rosas 2020',
                                  'Premios Juventud 2019', 'Feliz 2020', #'Amor Eterno',
                                  'Destino 2020 Presenta', 'Aqui y Ahora Censo 2020', 
                                  'Misa de Domingo de Ramos', 'Celebracion Eucaristica',
                                  'Teleton USA 2020', 'Noticias Univision Presenta']}

# Load Data

## Catalogues

In [8]:
%%time
## ID and name of contracts
id_contracts = pd.read_excel('CATALOGOS/ID_Contracts.xlsx')

## Time Zone
id_franja = pd.read_excel('CATALOGOS/ID_Franja.xlsx')

## ISCI, Duracion, Agrupacion & Pauta
id_ISCI = pd.read_excel('CATALOGOS/ID_ISCI.xlsx')

## Impressions by Chanel and time zone
id_imps = pd.read_excel('CATALOGOS/Historic_impresions_modif.xlsx')

Wall time: 6.07 s


## Weekly data

In [9]:
%%time
## Read Data and drop rows with all the columns with NA
raw = pd.read_excel('FILES/{0}/{1}'.format(quarter, filename))
raw.dropna(how='all', inplace=True)
raw.reset_index(drop=True, inplace=True)

Wall time: 3.45 s


In [10]:
raw['Unnamed: 0'].unique()

array(['GALA', nan, 'TUDN', 'UNIM', 'UNIV'], dtype=object)

In [11]:
raw.loc[raw['Unnamed: 0'] == ' ', 'Unnamed: 0'] = None

# Data Cleaning

## Raw data

In [12]:
## Fill cells with the value above it for the first 5 columns
for i in range(0,5):
    raw[raw.columns[i]] = raw[raw.columns[i]].fillna(method='ffill')

In [13]:
## Copy timezones into another column and erase them from the original
raw.loc[raw[raw.columns[6]].str.contains(':').fillna(False),raw.columns[5]] = raw.loc[raw[raw.columns[6]].str.contains(':').fillna(False),raw.columns[6]]
raw.loc[raw[raw.columns[6]].str.contains(':').fillna(False),raw.columns[6]] = None

In [14]:
## Fill cells again for the remaining columns 
for i in range(5,8):
    raw[raw.columns[i]] = raw[raw.columns[i]].fillna(method='ffill')

In [15]:
## Save clean dataframe
df = raw[raw['Date'].notnull()].copy()
df.reset_index(drop=True, inplace=True)

## Rename id contracts column and change type to integer
df[df.columns[3]] = df[df.columns[3]].astype('int64')
df.rename({df.columns[3]:'CONTRACT_ID', df.columns[7]:'PROGRAM_NAME'}, inplace=True, axis=1)
df['PROGRAM_NAME'] = df['PROGRAM_NAME'].apply(lambda x: str(x).strip())

In [16]:
## Delete rows where ISCI is null
df.dropna(subset=['House #'], inplace=True)
df.reset_index(drop=True, inplace=True)

In [17]:
raw[:3]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Date,Weekday,...,Rate,Line,Spot #,House #,Priority,MG,Order Product Description,AE,Order Start,Order End
0,GALA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
1,GALA,Genomma Labs USA,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
2,GALA,Genomma Labs USA,"Genomma Lab USA, LLC",NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


In [18]:
raw['Unnamed: 0'].unique()

array(['GALA', 'TUDN', 'UNIM', 'UNIV'], dtype=object)

**IMPORTANT:** The file should only contain dates and blank cells on column "DATE", if there's something else please erase it otherwise the program won't run properly make sure to check it before running the code.

In [19]:
## Date manipulation
if 'Genomma Logs Week' in filename:
    df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
df['Day of Week'] = df['Date'].apply(lambda x: x.strftime("%A"))
df['Day number'] = df['Day of Week'].map(day_numbr_n)
df['DESC_SEM'] = df['Day of Week'].map(day_numbr)
df['Month'] = df['Date'].dt.month
df['DESC_MES'] = df['Month'].map(month_name)

## Merges

### Week & Year GenommaLab

In [20]:
df = pd.merge(df, df_tiempo, on='Date', how='left')

In [21]:
df.isnull().sum()[df.isnull().sum() > 1]

Unnamed: 6    927
Feature       927
MG            927
dtype: int64

In [22]:
df['Year'] = df['TmpAnioSemanaGenomma'] #Takes year from date
df['Quarter'] = df['TmpSemanaAnioGenomma'].apply(lambda x: 1 if x in range(0,14) else (2 if x in range(14,27) else (3 if x in range(27,40) else 4)))
df['YEAR_WEEK'] = df['TmpAnioSemanaGenomma'].astype(str) + '-' + df['TmpSemanaAnioGenomma'].astype(str)

## Change lenght to number
df['Length'] = df['Length'].apply(lambda x: int(x.split(':')[1]) if x.split(':')[0] == '' else int(x.split(':')[0])*60 + int(x.split(':')[1])) 

In [23]:
## AirTime manipulation
df['Air Time'] = df['Air Time'].apply(lambda x: x.replace('XM','AM'))
df['Air Time Nmbr'] = df['Air Time'].apply(lambda x: x.split(' ')[0]); df['Air Time Nmbr'] = pd.to_datetime(df['Air Time Nmbr'], format='%H:%M:%S').dt.time
df['Hour'] = df['Air Time Nmbr'].apply(lambda x: x.hour)
df['TimeZone'] = df['Air Time'].apply(lambda x: x.split(' ')[1])

## Change to 24 hour format
df['Air Time 24'] = pd.to_datetime(df['Air Time'], format='%I:%M:%S %p').dt.time
df['Hour24'] = df['Air Time 24'].apply(lambda x: x.hour)
df['HOUR_ID'] = df['Hour'].astype(str) + '-' + df['TimeZone']
df['FRANJA_ID'] = df['DESC_SEM'] + '-' + df['Hour24'].astype(str)

In [24]:
## Remove some useless columns
df.drop(['Feature','MG'], axis=1,inplace=True)

### Contracts

In [25]:
id_contracts['CONTRACT_ID'].value_counts()[id_contracts['CONTRACT_ID'].value_counts() > 1]

Series([], Name: CONTRACT_ID, dtype: int64)

In [26]:
df = pd.merge(df, id_contracts, how='left', on='CONTRACT_ID')

In [27]:
df.isnull().sum()[df.isnull().sum() > 1]

Unnamed: 6    927
dtype: int64

In [28]:
id_franja['FRANJA_ID'].value_counts()[id_franja['FRANJA_ID'].value_counts() > 1]

Series([], Name: FRANJA_ID, dtype: int64)

### Timezones

In [29]:
df['franja_grouped'] = df['Hour24'].apply(lambda x: 'Day' if x in list(range(6,19)) else ('Prime' if x in list(range(19,23)) else 'Late'))
df = pd.merge(df, id_franja, how='left', on='FRANJA_ID')

In [30]:
df.isnull().sum()[df.isnull().sum() > 1]

Unnamed: 6    927
dtype: int64

### Impresions

In [31]:
## Change special dayparts of univision
df.reset_index(drop=True,inplace=True)
df['desc_franja_orig'] = df['DESC_FRANJA']
for i in ['Sports','News','Specials']:
    df_aux = df[(df.DESC_C.isin(['UNV TIER I','UNV TIER II','UNV TIER III'])) & (df.PROGRAM_NAME.isin(different_dayparts[i]))].copy()
    if df_aux.shape[0] == 0:
        continue
    df_aux['DESC_FRANJA'] = i
    df.drop(df_aux.index,axis=0,inplace=True)
    df = pd.concat([df,df_aux],axis=0)

In [32]:
id_imps['IMP_ID'].value_counts()[id_imps['IMP_ID'].value_counts() > 1]

Series([], Name: IMP_ID, dtype: int64)

In [33]:
df['IMP_ID'] = df['Year'].astype(str) + '-' + df['Quarter'].astype(str) + '-' + df['DESC_C'] + '-' + df['DESC_FRANJA']
df = pd.merge(df, id_imps.drop(['DESC_CON','QUARTER'], axis=1), how ='left', on='IMP_ID')

In [34]:
df.isnull().sum()[df.isnull().sum() > 1]

Unnamed: 6    927
dtype: int64

### ISCI

In [35]:
id_ISCI['ISCI'].value_counts()[id_ISCI['ISCI'].value_counts() > 1]

Series([], Name: ISCI, dtype: int64)

In [36]:
df.rename({'House #':'ISCI'},axis=1, inplace=True)
df = pd.merge(df, id_ISCI, how = 'left', on = 'ISCI')

In [37]:
df.isnull().sum()[df.isnull().sum() > 1]

Unnamed: 6    927
dtype: int64

### Agrupación, Pauta

In [38]:
df_groups = df_groups[~(df_groups['ProID']==2016)]
df_groups.reset_index(drop=True, inplace=True)

In [39]:
df_groups['AGRUPACION'].value_counts()[df_groups['AGRUPACION'].value_counts() > 1]

Series([], Name: AGRUPACION, dtype: int64)

In [40]:
df.loc[df['AGRUPACION']=='GOICOECHEA LINEA - USA', 'AGRUPACION'] = 'GOICOECHEA LINEA BASE - USA'

In [41]:
df = pd.merge(df, df_groups, how = 'left', on = 'AGRUPACION')

In [42]:
df.isnull().sum()[df.isnull().sum() > 1]

Unnamed: 6    927
dtype: int64

In [43]:
df['PAUTA'] = df['PAUTA'].apply(lambda x: str(x).strip())

# Final Transformations

In [44]:
# CALCULATE TRP'S
df['TRPS'] = df['IMP'] / 251529.5

df['Unidades'] = 1
df['Unidades Est.'] = df['Length'] / 30

df['TRPS Est.'] = df['TRPS'] * df['Unidades Est.']
df['IMP Est.'] = df['IMP'] * df['Unidades Est.']

#IDK WTF IS THIS COLUMN FOR
df['proveedor'] = 'No Info'

df_aux = df.pivot_table(index=['MrcNombre', 'IMP_ID'],values=['TRPS', 'TRPS Est.'], aggfunc='sum').reset_index()

df_aux.to_excel('Outputs/PT ' + filename[-13:-5] + '.xlsx', index=False)

In [45]:
cols_rev = ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'CONTRACT_ID', 'Unnamed: 4',
            'Unnamed: 5', 'Unnamed: 6', 'PROGRAM_NAME', 'Date', 'Weekday',
            'Air Time', 'Length', 'Inv. Code (Placed*)', 'State', 'Rate', 'Line',
            'Spot #', 'ISCI', 'Priority', 'Order Product Description', 'AE',
            'Order Start', 'Order End', 'Day of Week', 'Day number', 'DESC_SEM',
            'Year', 'Quarter', 'Month', 'DESC_MES', 'TmpSemanaAnioGenomma', 
            'TmpAnioSemanaGenomma', 'YEAR_WEEK', 'Air Time Nmbr', 'Hour', 'TimeZone',
            'Air Time 24', 'Hour24', 'HOUR_ID', 'FRANJA_ID', 'CONTRACT_H',
            'CONTRACT_TYPE', 'DESC_C', 'DESC_FRANJA', 'IMP_ID', 'CANAL', 'FRANJA', 
            'IMP', 'VERSION', 'DURACION', 'PAUTA', 'AGRUPACION', 'PAUTA DIRIGIDA', 
            'MrcID', 'MrcNombre', 'LinID', 'LinNombre', 'ProID', 'TRPS', 'Unidades',
            'Unidades Est.', 'TRPS Est.', 'IMP Est.']

cols_hist = ['franja_grouped', 'AGRUPACION', 'ProID', 'YEAR_WEEK', 'Air Time', 'proveedor',
             'CONTRACT_H', 'ISCI', 'Length', 'Date', 'TRPS', 'IMP', 'CONTRACT_TYPE', 'Hour24',
             'DESC_SEM', 'Year', 'TmpSemanaAnioGenomma', 'Month', 'DESC_MES', 'Quarter', 
             'VERSION', 'Day number', 'PAUTA DIRIGIDA', 'TRPS Est.',  'Unidades', 'Unidades Est.', 
             'IMP Est.', 'MrcNombre']

In [46]:
df_review = df[cols_rev].copy()

df_historic = df[cols_hist].copy()

# Validations

In [47]:
df_historic.isnull().sum()[df_historic.isnull().sum() > 1]

Series([], dtype: int64)

In [48]:
df_review.isnull().sum()[df_review.isnull().sum() > 1]

Unnamed: 6    927
dtype: int64

In [49]:
df_historic.pivot_table(index='MrcNombre',values=['TRPS', 'TRPS Est.'], aggfunc='sum')

,TRPS,TRPS Est.
MrcNombre,,
ASEPXIA,80.77,56.41
CICATRICURE,181.47,128.30
LOMECAN V,23.68,23.68
POMADA DEL DRAGON,28.04,28.04
SILKA,12.37,12.37
TIO NACHO,129.83,64.91
TUKOL,54.52,27.26
X RAY,37.34,37.34


In [50]:
df_historic[['TRPS', 'TRPS Est.']].sum()

TRPS        548.01
TRPS Est.   378.31
dtype: float64

# Export Data

In [51]:
year = (datetime.today() - timedelta(days=2)).year
week = datetime.today().isocalendar()[1]
week = str(week).zfill(2)

path = 'Outputs/{0}/W {1}/'.format(year, week) 
if not os.path.exists(path):
    os.mkdir(path)

df_historic.to_csv(path + 'Historic_media_w{1}.csv'.format(year, week), index=False, encoding='latin1')
df_review.to_csv(path + 'Raw_media_W{1}.csv'.format(year, week), index=False, encoding='latin1')

df_all = pd.DataFrame()
files_raros = []

for name in file_names:    
    try:
        ###############################
        #LOAD DATA
        #Read support tables
        id_contracts = pd.read_excel('CATALOGOS/ID_Contracts.xlsx')
        id_franja = pd.read_excel('CATALOGOS/ID_Franja.xlsx')
        id_ISCI = pd.read_excel('CATALOGOS/ID_ISCI.xlsx')
        id_imps = pd.read_excel('CATALOGOS/Historic_impresions.xlsx')
        #id_pauta = pd.read_excel('CATALOGOS/ID_PAUTA_DIRIGIDA_2.xlsx')
        #id_agrup = pd.read_excel('CATALOGOS/ID_AGRUPACIONES.xlsx')

        #Read Data and drop rows with all the columns with NA
        raw = pd.read_excel('FILES/{0}/{1}'.fomart(quarter, name))
        raw.dropna(how='all', inplace=True)
        raw.reset_index(drop=True, inplace=True)

        ###############################
        #PREPARE DATA
        #Fill cells with the value above it for the first 5 columns
        for i in range(0,5):
            raw[raw.columns[i]] = raw[raw.columns[i]].fillna(method='ffill')

        #Copy timezones into another column and erase them from the original
        raw.loc[raw[raw.columns[6]].str.contains(':').fillna(False),raw.columns[5]] = raw.loc[raw[raw.columns[6]].str.contains(':').fillna(False),raw.columns[6]]
        raw.loc[raw[raw.columns[6]].str.contains(':').fillna(False),raw.columns[6]] = None

        #Fill cells again for the remaining columns 
        for i in range(5,8):
            raw[raw.columns[i]] = raw[raw.columns[i]].fillna(method='ffill')

        #Save clean dataframe
        df = raw[raw['Date'].notnull()].reset_index(drop=True)
        df[df.columns[3]] = df[df.columns[3]].astype('int64'); df.rename({df.columns[3]:'CONTRACT_ID',df.columns[7]:'PROGRAM_NAME'},inplace=True,axis=1) #Rename id contracts column and change type to integer
        df['PROGRAM_NAME'] = df['PROGRAM_NAME'].apply(lambda x: str(x).strip())

        df_raw = df.copy()

        ###############################
        #DATAFRAME MANIPULATION
        #Delete rows where ISCI is null
        df.dropna(subset=['House #'],inplace=True)
        df.reset_index(drop=True,inplace=True)

        #Date manipulation
        if 'Genomma Logs Week' in name:
            df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%y')
        df['Day of Week'] = df['Date'].apply(lambda x: x.strftime("%A"))
        df['Day number'] = df['Day of Week'].map(day_numbr_n)
        df['DESC_SEM'] = df['Day of Week'].map(day_numbr)
        df['Month'] = df['Date'].dt.month
        df['DESC_MES'] = df['Month'].map(month_name)

        #Relate week and year Genomma
        df = pd.merge(df,df_tiempo,on='Date',how='left')
        df['Year'] = df['TmpAnioSemanaGenomma'] #Takes year from date
        df['Quarter'] = df['TmpSemanaAnioGenomma'].apply(lambda x: 1 if x in range(0,14) else (2 if x in range(14,27) else (3 if x in range(27,40) else 4)))
        df['YEAR_WEEK'] = df['TmpAnioSemanaGenomma'].astype(str) + '-' + df['TmpSemanaAnioGenomma'].astype(str)

        df['Length'] = df['Length'].apply(lambda x: int(x.split(':')[1]) if x.split(':')[0] == '' else int(x.split(':')[0])*60 + int(x.split(':')[1])) #Change lenght to number

        #AirTime manipulation
        df['Air Time'] = df['Air Time'].apply(lambda x: x.replace('XM','AM'))
        df['Air Time Nmbr'] = df['Air Time'].apply(lambda x: x.split(' ')[0]); df['Air Time Nmbr'] = pd.to_datetime(df['Air Time Nmbr'], format='%H:%M:%S').dt.time
        df['Hour'] = df['Air Time Nmbr'].apply(lambda x: x.hour)
        df['TimeZone'] = df['Air Time'].apply(lambda x: x.split(' ')[1])
        df['Air Time 24'] = pd.to_datetime(df['Air Time'], format='%I:%M:%S %p').dt.time #Change to 24 hour format
        df['Hour24'] = df['Air Time 24'].apply(lambda x: x.hour)
        df['HOUR_ID'] = df['Hour'].astype(str) + '-' + df['TimeZone']
        df['FRANJA_ID'] = df['DESC_SEM'] + '-' + df['Hour24'].astype(str)

        #Remove some useless columns
        df.drop(['Feature','MG'], axis=1,inplace=True)

        ###############################
        #RELATE CONTRACTS
        df = pd.merge(df,id_contracts,how='left', on='CONTRACT_ID')

        ###############################
        #RELATE TIMEZONES
        df['franja_grouped'] = df['Hour24'].apply(lambda x: 'Day' if x in list(range(6,19)) else ('Prime' if x in list(range(19,23)) else 'Late'))
        df = pd.merge(df,id_franja,how='left', on='FRANJA_ID')

        ###############################
        #RELATE IMPRESIONS
        #Change special dayparts of univision
        df.reset_index(drop=True,inplace=True)
        df['desc_franja_orig'] = df['DESC_FRANJA']
        for i in ['Sports','News','Specials']:
            df_aux = df[(df.DESC_C.isin(['UNV TIER I','UNV TIER II','UNV TIER III'])) & (df.PROGRAM_NAME.isin(different_dayparts[i]))].copy()
            if df_aux.shape[0] == 0:
                continue
            df_aux['DESC_FRANJA'] = i
            df.drop(df_aux.index,axis=0,inplace=True)
            df = pd.concat([df,df_aux],axis=0)

        df['IMP_ID'] = df['Year'].astype(str) + '-' + df['Quarter'].astype(str) + '-' + df['DESC_C'] + '-' + df['DESC_FRANJA']
        df = pd.merge(df, id_imps.drop(['DESC_CON','QUARTER'],axis=1), how = 'left', on = 'IMP_ID')

        ###############################
        #RELATE ISCI, PAUTA, ETC
        df.rename({'House #':'ISCI'},axis=1, inplace=True)
        df = pd.merge(df, id_ISCI, how = 'left', on = 'ISCI')

        df['PAUTA'] = df['PAUTA'].apply(lambda x: str(x).strip())
        #df = pd.merge(df, id_pauta, how = 'left', on = 'PAUTA')

        df = pd.merge(df, df_groups, how = 'left', on = 'AGRUPACION')

        ###############################
        #CALCULATE TRP'S
        df['TRPS'] = df['IMP'] / 251529.5

        df['Unidades'] = 1
        df['Unidades Est.'] = df['Length'] / 30

        df['TRPS Est.'] = df['TRPS'] * df['Unidades Est.']
        df['IMP Est.'] = df['IMP'] * df['Unidades Est.']

        #IDK WTF IS THIS COLUMN FOR
        df['proveedor'] = 'No Info'

        ###############################
        #OUTPUTS
        df.columns = [i.lower() for i in df.columns]

        df_all = pd.concat([df_all,df])
        print(name)

    except:
        files_raros.append(name)
        continue
        
#if df_all[df_all['agrupacion'].str.contains('LINEA')].shape[0] > 0:
    
     #print('Se encontró un spot dirigido a una linea completa...')
     #df_all = linea_treatment(df_all)
    
df_review = df_all[['unnamed: 0','unnamed: 1','unnamed: 2','contract_id','unnamed: 4','unnamed: 5','unnamed: 6','program_name','date','weekday','air time','length','inv. code (placed*)','state','rate','line','spot #','isci','priority','order product description','ae','order start','order end','day of week','day number','desc_sem','year','quarter','month','desc_mes','tmpsemanaaniogenomma','tmpaniosemanagenomma','year_week','air time nmbr','hour','timezone','air time 24','hour24','hour_id','franja_id','contract_h','contract_type','desc_c','desc_franja','imp_id','canal','franja','imp','version','duracion','pauta','agrupacion','pauta dirigida','mrcid','mrcnombre','linid','linnombre','proid','trps','unidades','unidades est.','trps est.','imp est.']].copy()
df_historic = df_all[['franja_grouped','agrupacion','proid','year_week','air time','proveedor','contract_h','isci','length','date','trps','imp','contract_type','hour24','desc_sem','year','tmpsemanaaniogenomma','month','desc_mes','quarter','version','day number','pauta dirigida','trps est.','unidades','unidades est.','imp est.','mrcnombre']].copy()
